In [140]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.display import display

# Data Pemakaian Listrik Mingguan dan Faktor Eksogen
data = {
    'Minggu': ['Week 1', 'Week 2', 'Week 3', 'Week 4',
               'Week 5', 'Week 6', 'Week 7', 'Week 8'],
    'Banda Sakti': [150, 160, 155, 165, 170, 175, 180, 185],
    'Blang Mangat': [200, 210, 205, 220, 225, 230, 235, 240],
    'Muara Satu': [130, 135, 140, 145, 150, 155, 160, 165],
    'Muara Dua': [180, 185, 190, 195, 200, 205, 210, 215],
    'Suhu (X1)': [20, 21, 19, 22, 20, 23, 24, 22],  # Faktor Eksogen 1
    'Harga Listrik per Watt (X2)': [5.5, 5.4, 5.6, 5.5,
                                    5.7, 5.8, 5.9, 5.6]  # Faktor Eksogen 2
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Tampilkan DataFrame
display(df)


,Minggu,Banda Sakti,Blang Mangat,Muara Satu,Muara Dua,Suhu (X1),Harga Listrik per Watt (X2)
0,Week 1,150,200,130,180,20,5.5
1,Week 2,160,210,135,185,21,5.4
2,Week 3,155,205,140,190,19,5.6
3,Week 4,165,220,145,195,22,5.5
4,Week 5,170,225,150,200,20,5.7
5,Week 6,175,230,155,205,23,5.8
6,Week 7,180,235,160,210,24,5.9
7,Week 8,185,240,165,215,22,5.6


In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Parameter untuk model VARMA
phi_varma = 1.0298
theta_varma = 4.445
beta1 = 0.3
beta2 = 0.2

# Daftar wilayah
regions = ['Banda Sakti', 'Blang Mangat', 'Muara Satu', 'Muara Dua']

# Menyimpan DataFrame per wilayah
df_regions = {}

# Metrik kesalahan per wilayah
error_metrics = {}

# Definisikan warna untuk model
colors = {
    'Pemakaian Asli': 'blue',
    'MA (3-Mingguan)': 'orange',
    'AR(1)': 'green',
    'AR+MA': 'red',
    'VARMA': 'purple'
}

for region in regions:
    # Membuat salinan DataFrame untuk setiap wilayah
    df_region = df[['Minggu', region, 'Suhu (X1)', 'Harga Listrik per Watt (X2)']].copy()
    df_region.rename(columns={region: 'Y'}, inplace=True)
    
    # 1. Menghitung Moving Average (MA) 3-mingguan
    def moving_average(series, period=3):
        return series.rolling(window=period).mean()
    
    df_region['MA (3-Mingguan)'] = moving_average(df_region['Y'])
    
    # 2. Menghitung model Autoregressive (AR)
    phi_ar = 1.0298  # Nilai phi untuk model AR(1)
    df_region['AR(1)'] = df_region['Y'].shift(-1) * phi_ar
    
    # 3. Menghitung model AR + MA
    df_region['Residuals'] = df_region['Y'] - df_region['MA (3-Mingguan)']
    df_region['Residuals'].fillna(0, inplace=True)  # Mengganti NaN dengan 0
    
    df_region['AR+MA'] = (phi_varma * df_region['Y'])+(df_region['Residuals'].shift(-1) * theta_varma)  # Menggunakan residual H-1
    df_region['AR+MA'].fillna(0, inplace=True)  # Mengganti NaN dengan 0
    
    # 4. Menghitung model VARMA(1,1)
    def calculate_varma_values(series, residuals, phi, theta, x1, x2, beta1, beta2):
        varma_values = [None, None]  # VARMA tidak dapat dihitung untuk 2 minggu pertama
        for t in range(2, len(series)):
            varma_value = ((phi * series[t-2]) + theta * residuals[t-2] + (
                           beta1 * x1[t]) + beta2 * x2[t])
            varma_values.append(varma_value)
        return varma_values
    
    df_region['VARMA'] = calculate_varma_values(
        df_region['Y'].shift(-1), df_region['Residuals'].shift(-2), phi_varma, theta_varma,
        df_region['Suhu (X1)'], df_region['Harga Listrik per Watt (X2)'], beta1, beta2)
    
    # 5. Menghitung MAPE, RMSE, dan MAE
    error_metrics_region = {}
    models = ['MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA']
    for model in models:
        actual = df_region['Y'][df_region[model].notna()]
        predicted = df_region[model][df_region[model].notna()]
        mae = mean_absolute_error(actual, predicted)
        rmse = mean_squared_error(actual, predicted, squared=False)
        mape = np.mean(np.abs((actual - predicted) / actual)) * 100
        error_metrics_region[model] = {'MAE': round(mae, 2), 'RMSE': round(rmse, 2), 'MAPE': round(mape, 2)}
    error_metrics[region] = error_metrics_region
    
    # Menyimpan df_region
    df_regions[region] = df_region.copy()

# Menampilkan metrik kesalahan untuk setiap wilayah
for region in regions:
    print(f"\nMetrik Kesalahan untuk {region}:")
    error_df_region = pd.DataFrame(error_metrics[region]).T
    display(error_df_region)


5.6
5.5
5.7
5.8
5.9
5.6
5.6
5.5
5.7
5.8
5.9
5.6
5.6
5.5
5.7
5.8
5.9
5.6
5.6
5.5
5.7
5.8
5.9
5.6

Metrik Kesalahan untuk Banda Sakti:


,MAE,RMSE,MAPE
MA (3-Mingguan),4.44,4.91,2.55
AR(1),10.17,11.11,6.23
AR+MA,42.24,69.32,23.91
VARMA,28.22,28.91,16.33



Metrik Kesalahan untuk Blang Mangat:


,MAE,RMSE,MAPE
MA (3-Mingguan),5.28,5.97,2.31
AR(1),12.38,13.62,5.74
AR+MA,53.28,89.42,23.10
VARMA,32.68,33.67,14.41



Metrik Kesalahan untuk Muara Satu:


,MAE,RMSE,MAPE
MA (3-Mingguan),5.00,5.00,3.29
AR(1),9.47,9.47,6.55
AR+MA,41.07,62.74,26.45
VARMA,29.26,29.27,19.22



Metrik Kesalahan untuk Muara Dua:


,MAE,RMSE,MAPE
MA (3-Mingguan),5.00,5.00,2.47
AR(1),10.96,10.96,5.63
AR+MA,48.63,79.84,23.56
VARMA,30.75,30.76,15.20


In [142]:
# 7. Plotting grafik perbandingan per wilayah dalam grid 2x2 menggunakan Plotly
fig = make_subplots(rows=2, cols=2, subplot_titles=regions)

for idx, region in enumerate(regions):
    df_region = df_regions[region]
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # Pemakaian Asli
    fig.add_trace(go.Scatter(x=df_region['Minggu'], y=df_region['Y'], mode='lines+markers', 
                             name='Pemakaian Asli', line=dict(color=colors['Pemakaian Asli'])), row=row, col=col)
    
    # Menambahkan model dengan warna yang ditentukan
    for model in models:
        show_legend = (region == regions[0])  # Hanya tunjukkan legend pada subplot pertama
        fig.add_trace(go.Scatter(x=df_region['Minggu'], y=df_region[model], mode='lines+markers', 
                                 name=model if show_legend else '', line=dict(color=colors[model]), showlegend=show_legend), row=row, col=col)

# Menyesuaikan layout
fig.update_layout(title_text='Perbandingan Pemakaian Listrik per Wilayah', height=800)
fig.update_xaxes(title_text='Minggu')
fig.update_yaxes(title_text='Pemakaian Listrik (mWh)')
fig.show()


In [143]:
# Menggabungkan semua wilayah dalam satu DataFrame untuk ditampilkan
combined_results = pd.DataFrame(columns=['Minggu', 'Wilayah', 'Pemakaian Asli', 'MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA'])

for region in regions:
    df_region = df_regions[region][['Minggu', 'Y', 'MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA']].copy()
    df_region['Wilayah'] = region
    df_region.rename(columns={'Y': 'Pemakaian Asli'}, inplace=True)
    combined_results = pd.concat([combined_results, df_region], ignore_index=True)

# Mengatur format desimal
combined_results = combined_results.round(3)

# Menampilkan DataFrame gabungan
display(combined_results)


C:\Users\User\AppData\Local\Temp\ipykernel_14000\741637472.py:8: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,Minggu,Wilayah,Pemakaian Asli,MA (3-Mingguan),AR(1),AR+MA,VARMA
0,Week 1,Banda Sakti,150,NaN,164.768,154.470,NaN
1,Week 2,Banda Sakti,160,NaN,159.619,164.768,NaN
2,Week 3,Banda Sakti,155,155.000,169.917,181.844,171.588
3,Week 4,Banda Sakti,165,160.000,175.066,199.550,189.544
4,Week 5,Banda Sakti,170,163.333,180.215,197.291,206.690
5,Week 6,Banda Sakti,175,170.000,185.364,202.440,205.351
6,Week 7,Banda Sakti,180,175.000,190.513,207.589,210.820
7,Week 8,Banda Sakti,185,180.000,NaN,0.000,215.309
8,Week 1,Blang Mangat,200,NaN,216.258,205.960,NaN
9,Week 2,Blang Mangat,210,NaN,211.109,216.258,NaN
